In [2]:

import datetime as dt
import pandas as pd
import numpy as np
import requests
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import h2o
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.under_sampling import RandomUnderSampler, TomekLinks
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from math import radians, sin, cos, sqrt, atan2
from shapely.geometry import Polygon
from sklearn.preprocessing import LabelEncoder
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from statsmodels.tools.tools import add_constant

from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LinearRegression
from h2o.automl import H2OAutoML
from tpot import TPOTClassifier
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import xgboost
from sklearn.metrics import mean_squared_error, r2_score, roc_auc_score
from sklearn.linear_model import LinearRegression 
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVC


In [3]:
accidents_sub = pd.read_csv("C:\\Users\\reza3\\OneDrive\\Desktop\\AIT\\Machine learning\\group project\\data\\sub_accidents.csv")

In [9]:
accidents_sub = accidents_sub[accidents_sub["Start_Year"]>2020]

In [12]:
accidents_sub.shape

(3047692, 37)

In [14]:
X = accidents_sub.drop(columns=['ID', 'Severity', 'Delay(min)', 'Street', 'City', 'County', 'State', 'Zipcode',
       'Start_Year', 'Start_Month', 'Start_Day', 'Start_Lat', 'Start_Lng',
       'Start_time','Distance(km)', 'Delay_ln', 'Severity_new'])
Y = accidents_sub.loc[:, ['Severity', 'Severity_new', 'Delay(min)', 'Delay_ln', 'Distance(km)']]


In [19]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y["Severity_new"], test_size = 0.10, random_state=370, stratify=Y["Severity_new"])

assert len(X_train)  == len(Y_train)
assert len(X_test)   == len(Y_test)
X.shape, Y.shape,X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((3047692, 20),
 (3047692, 5),
 (2742922, 20),
 (304770, 20),
 (2742922,),
 (304770,))

In [ ]:
'''# fixing imbalance by under sampling (Tomek links) (it didn't help)
tl = TomekLinks(n_jobs=-1, sampling_strategy="Majority")

# fit predictor and target variable
X_train_tl, Y_train_tl = tl.fit_resample(X_train, Y_train["Severity_new"])'''

In [20]:
# fixing imbalance by under sampling (random under sampling)
rus = RandomUnderSampler(random_state=370, replacement=False)

# fit predictor and target varialbe
X_train_rus, Y_train_rus = rus.fit_resample(X_train, Y_train)

X.shape, Y.shape, X_train_rus.shape, Y_train_rus.shape

((3047692, 20), (3047692, 5), (490126, 20), (490126,))

I tried to fit a regression model with Delay and distance as labels but no appropriate model have found.

In [ ]:
'''model = SGDRegressor(max_iter=1000, tol=1e-3, random_state=370)

model.fit(X_train, Y_train["Distance(km)"])

# Make predictions on the validation set
y_train_pred = model.predict(X_train)

# Calculate metrics
mse = mean_squared_error(Y_train["Distance(km)"], y_train_pred)
r2 = r2_score(Y_train["Distance(km)"], y_train_pred)
print(f'Mean Squared Error on Test Set: {mse}')
print(f'R_squared on Test Set: {r2}')'''




In [ ]:
device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
print("Device: ", device)

In [16]:
Y_train.value_counts()

Severity_new
0    2359090
1     231448
Name: count, dtype: int64

In [22]:
lr = LogisticRegression(random_state=370, n_jobs=-1)
rf = RandomForestClassifier(random_state=370, n_jobs=-1, oob_score=True)
sgb = xgboost.XGBClassifier(max_depth=1, n_estimators=200, n_jobs=-1)
ada = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), n_estimators=200, learning_rate=0.001, random_state=370)

models = [ lr, rf, sgb, ada]

#perform cross validation using KFold
from sklearn.model_selection import KFold, cross_val_score

kfold = KFold(n_splits = 5, shuffle = True, random_state=370)

for model in models:
    score = cross_val_score(model, X_train_rus, Y_train_rus, cv=kfold, scoring='accuracy', n_jobs=-1)  #f1, recall, precision, accuracy
    np.set_printoptions(precision = 2)
    print("Model: ", model, "Scores: ", score, "- Scores mean: ", score.mean(), "- Scores std (lower better):", score.std()) 
    fit = model.fit(X_train_rus, Y_train_rus)
    yhat = fit.predict(X_test)
    print(confusion_matrix(Y_test, yhat).ravel())
    print('ROCAUC score:',roc_auc_score(Y_test, yhat))      
    print(classification_report(Y_test, yhat))

Model:  LogisticRegression(n_jobs=-1, random_state=370) Scores:  [0.57 0.57 0.57 0.57 0.57] - Scores mean:  0.5707777215891273 - Scores std (lower better): 0.0016765492161903536
[125476 152065   8439  18790]
ROCAUC score: 0.5710860265028719
              precision    recall  f1-score   support

           0       0.94      0.45      0.61    277541
           1       0.11      0.69      0.19     27229

    accuracy                           0.47    304770
   macro avg       0.52      0.57      0.40    304770
weighted avg       0.86      0.47      0.57    304770

Model:  RandomForestClassifier(n_jobs=-1, oob_score=True, random_state=370) Scores:  [0.62 0.62 0.62 0.62 0.63] - Scores mean:  0.6241578733076416 - Scores std (lower better): 0.0012007246685853084
[171228 106313   9713  17516]
ROCAUC score: 0.6301157092717378
              precision    recall  f1-score   support

           0       0.95      0.62      0.75    277541
           1       0.14      0.64      0.23     27229

    acc

In [25]:
param_grid = {"n_estimators": [100, 200, 300], "criterion": ["gini", "entropy"], "max_depth": np.arange(1, 10)}
model = RandomForestClassifier(n_jobs=-1, random_state=370, oob_score=True)
grid = GridSearchCV(model, param_grid, refit=True, n_jobs=-1)
grid.fit(X_train_rus, Y_train_rus)
print(grid.best_params_)
yhat = grid.predict(X_test)
print(classification_report(Y_test, yhat))

{'criterion': 'gini', 'max_depth': 9, 'n_estimators': 200}
              precision    recall  f1-score   support

           0       0.94      0.57      0.71    277541
           1       0.13      0.64      0.21     27229

    accuracy                           0.58    304770
   macro avg       0.53      0.61      0.46    304770
weighted avg       0.87      0.58      0.66    304770



In [28]:
model = RandomForestClassifier(n_estimators= 200, criterion="gini", max_depth= 9, n_jobs=-1, random_state=370)
model.fit(X_train_rus, Y_train_rus)
yhat = model.predict(X_test)
tn, fp, fn, tp = confusion_matrix(Y_test, yhat).ravel()
print("True Negative = ",tn, " False Negative = ",fn, "True Positive = ", tp, "False Positive = ", fn)
print(classification_report(Y_test, yhat))

True Negative =  157810  False Negative =  9709 True Positive =  17520 False Positive =  9709
              precision    recall  f1-score   support

           0       0.94      0.57      0.71    277541
           1       0.13      0.64      0.21     27229

    accuracy                           0.58    304770
   macro avg       0.53      0.61      0.46    304770
weighted avg       0.87      0.58      0.66    304770



In [30]:
model = RandomForestClassifier(n_estimators= 200, criterion="gini", max_depth= 9, n_jobs=-1, random_state=370, class_weight="balanced_subsample")
model.fit(X_train, Y_train)
yhat = model.predict(X_test)
tn, fp, fn, tp = confusion_matrix(Y_test, yhat).ravel()
print("True Negative = ",tn, " False Negative = ",fn, "True Positive = ", tp, "False Positive = ", fn)
print(classification_report(Y_test, yhat))

True Negative =  159513  False Negative =  9900 True Positive =  17329 False Positive =  9900
              precision    recall  f1-score   support

           0       0.94      0.57      0.71    277541
           1       0.13      0.64      0.21     27229

    accuracy                           0.58    304770
   macro avg       0.53      0.61      0.46    304770
weighted avg       0.87      0.58      0.67    304770



In [31]:
model = RandomForestClassifier(n_estimators= 200, criterion="gini", max_depth= 9, n_jobs=-1, random_state=370, class_weight="balanced")
model.fit(X_train, Y_train)
yhat = model.predict(X_test)
tn, fp, fn, tp = confusion_matrix(Y_test, yhat).ravel()
print("True Negative = ",tn, " False Negative = ",fn, "True Positive = ", tp, "False Positive = ", fn)
print(classification_report(Y_test, yhat))